<a href="https://colab.research.google.com/github/deacs11/CrewAI_Personalized_Learning_Path/blob/main/CrewAI_Personalized_Learning_Path.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 1. Install Necessary Libraries
# Installs crewai, tools, OpenAI client, Colab module, and search tool
!pip install crewai crewai-tools langchain-openai google-colab duckduckgo-search==5.3.1b1 -q
!pip install --upgrade crewai

print("Library installation completed!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 510.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s 

In [2]:
# @title 2. Import Modules and Configure API Keys from Secrets
import os
from google.colab import userdata # To read Colab secrets
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool # Using Serper for web search
from langchain_openai import ChatOpenAI

# --- API KEY CONFIGURATION FROM COLAB SECRETS ---
print("Attempting to load API Keys from Secrets...")
try:
    # Reads the OpenAI key from the secret 'OPENAI_API_KEY'
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    # Reads the Serper key from the secret 'SERPER_API_KEY'
    os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
    print("-> API Key environment variables set successfully.")
except Exception as e:
    print(f"!!! Error loading API Keys from Secrets: {e}")
    print("!!! Make sure you have set them correctly in the Colab Secrets panel")
    print("!!! and enabled 'Notebook access' for both.")

# Debug block to verify keys
print("-" * 20)
retrieved_key = os.environ.get("OPENAI_API_KEY")
if retrieved_key:
    print(f"OpenAI Key FOUND in environment: '{retrieved_key[:5]}...{retrieved_key[-4:]}'")
else:
    print("!!! OpenAI Key NOT FOUND.")
print("-" * 20)
retrieved_serper_key = os.environ.get("SERPER_API_KEY")
if retrieved_serper_key:
     print(f"Serper Key FOUND in environment: '{retrieved_serper_key[:5]}...{retrieved_serper_key[-4:]}'")
else:
    print("--- Serper Key NOT FOUND (required for ResourceFinder Agent).")
print("-" * 20)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_t

Attempting to load API Keys from Secrets...
-> API Key environment variables set successfully.
--------------------
OpenAI Key FOUND in environment: 'sk-pr...DKkA'
--------------------
Serper Key FOUND in environment: '7354c...dc56'
--------------------


In [3]:
# @title 3. Define User Input (Goal, Knowledge, Style)

# --- MODIFY THESE VARIABLES WITH THE USER'S DETAILS, THIS IS JUST AN EXAMPLE ---
learning_goal = "Learn Python programming for data analysis and basic machine learning tasks."

current_knowledge = """
I have some basic programming experience with JavaScript (variables, loops, basic functions).
I understand fundamental concepts but have never used Python before.
I am completely new to data science libraries like Pandas, NumPy, or Scikit-learn.
I am comfortable using a computer and searching online.
"""

preferred_style = """
I learn best through a combination of:
1.  Hands-on projects where I can apply concepts immediately.
2.  Clear video tutorials explaining concepts visually.
3.  Reading documentation or articles for specific details or deeper dives, but not as the primary method.
I prefer resources that are practical and goal-oriented. Avoid purely theoretical resources initially.
"""
# ---------------------------------------------------------------------

print("User Input Defined:")
print(f"Learning Goal: {learning_goal}")
print(f"Current Knowledge: {current_knowledge.strip()}")
print(f"Preferred Style: {preferred_style.strip()}")

User Input Defined:
Learning Goal: Learn Python programming for data analysis and basic machine learning tasks.
Current Knowledge: I have some basic programming experience with JavaScript (variables, loops, basic functions).
I understand fundamental concepts but have never used Python before.
I am completely new to data science libraries like Pandas, NumPy, or Scikit-learn.
I am comfortable using a computer and searching online.
Preferred Style: I learn best through a combination of:
1.  Hands-on projects where I can apply concepts immediately.
2.  Clear video tutorials explaining concepts visually.
3.  Reading documentation or articles for specific details or deeper dives, but not as the primary method.
I prefer resources that are practical and goal-oriented. Avoid purely theoretical resources initially.


In [4]:
# @title 4. Select LLM and Initialize Tools

# --- CHOOSE THE LANGUAGE MODEL (LLM) ---
try:
    llm = ChatOpenAI(model="gpt-4-turbo") # More capable for nuanced tasks like planning
    # llm = ChatOpenAI(model="gpt-3.5-turbo") # Cheaper alternative
    print(f"LLM ({llm.model_name}) initialized successfully.")
except Exception as e:
    print(f"!!! Error initializing ChatOpenAI: {e}")
    print("!!! Verify OpenAI API key.")
    raise

# --- INITIALIZE TOOLS ---
# Web search tool is needed for the ResourceFinder Agent
search_tool = None
try:
    search_tool = SerperDevTool()
    print("SerperDevTool initialized.")
except Exception as e:
    print(f"--- Warning: Error initializing SerperDevTool: {e}. Web search for resources might fail.")

available_tools = [search_tool] if search_tool else []
print(f"Tools actually available: {[t.name for t in available_tools] if available_tools else 'None'}")

LLM (gpt-4-turbo) initialized successfully.
SerperDevTool initialized.
Tools actually available: ['Search the internet with Serper']


In [5]:
# @title 5. Define Agents for Learning Path Crew

print("Defining Learning Path Agents...")

if 'llm' not in locals() or llm is None:
     raise ValueError("LLM not initialized.")

# --- Agent 1: Goal Clarifier ---
goal_clarifier = Agent(
    role='Learning Goal Analyst',
    goal=f'Break down the main learning goal "{learning_goal}" into 3-5 core, logically ordered sub-topics or essential skill areas required to achieve it.',
    backstory=(
        "You are an educational expert specializing in curriculum design. Your strength lies in deconstructing complex learning objectives "
        "into smaller, manageable, and sequential modules suitable for beginners in the subject area. You focus on identifying the foundational "
        "pillars needed to build towards the final goal."
    ),
    tools=[], # Does not need external tools, relies on LLM reasoning
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=3
)
print("- Agent 'goal_clarifier' defined.")

# --- Agent 2: Knowledge Assessor ---
knowledge_assessor = Agent(
    role='Skill Assessment Specialist',
    goal=f'Based on the user\'s described current knowledge ("{current_knowledge}") and the list of required sub-topics (provided by the Goal Clarifier), identify the primary knowledge gaps. Focus on pinpointing which required sub-topics the user likely needs to start with or focus on most.',
    backstory=(
        "You are an experienced tutor adept at understanding a learner's starting point. You carefully read their self-assessment "
        "and compare it against the required curriculum (sub-topics) to highlight the specific areas where learning needs to begin. "
        "You provide a concise summary of these gaps."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=3
)
print("- Agent 'knowledge_assessor' defined.")

# --- Agent 3: Resource Finder ---
resource_finder = Agent(
    role='Educational Resource Curator',
    goal=f'Find 1-2 high-quality, relevant, and generally well-regarded online learning resources for *each* sub-topic identified by the Goal Clarifier. Prioritize resources matching the user\'s preferred style: "{preferred_style}". Resources can include specific URLs to tutorials, videos, interactive platforms, key documentation pages, or project idea repositories.',
    backstory=(
        "You are the librarian for the digital age. You excel at using web search tools to uncover effective and reputable learning materials. "
        "You filter search results based on relevance to the sub-topic and alignment with the learner's preferred styles (e.g., videos, projects). "
        "You aim for quality over quantity, providing direct links where possible."
    ),
    tools=available_tools, # Requires search_tool
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=7 # May need more iterations for searching multiple topics
)
print("- Agent 'resource_finder' defined.")

# --- Agent 4: Path Planner ---
path_planner = Agent(
    role='Learning Path Designer',
    goal='Sequence the identified sub-topics (from Goal Clarifier) and their corresponding resources (from Resource Finder) into a logical, step-by-step learning path. Start addressing the knowledge gaps (from Knowledge Assessor) first. Suggest a practical order for learning, potentially grouping related resources or suggesting small milestones.',
    backstory=(
        "You are an instructional designer focused on creating effective and engaging learning journeys. You understand the importance of prerequisite knowledge "
        "and structure learning progressively, building complexity step-by-step. You organize topics and resources into a coherent flow designed to take the learner "
        "from their starting point towards their goal."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=5
)
print("- Agent 'path_planner' defined.")

# --- Agent 5: Plan Presenter ---
plan_presenter = Agent(
    role='Learning Plan Communicator',
    goal='Format the sequenced learning path (created by the Path Planner) into a clear, actionable, encouraging, and easy-to-follow final document for the user. Use clear headings, bullet points, and include the curated resource links.',
    backstory=(
        "You are an excellent communicator skilled at presenting structured information in a user-friendly and motivating format. "
        "You take the logical plan and turn it into a polished final output that the learner can directly use to start their journey. "
        "Clarity and actionability are your priorities."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=3
)
print("- Agent 'plan_presenter' defined.")

print("All learning path agents defined.")

Defining Learning Path Agents...
- Agent 'goal_clarifier' defined.
- Agent 'knowledge_assessor' defined.
- Agent 'resource_finder' defined.
- Agent 'path_planner' defined.
- Agent 'plan_presenter' defined.
All learning path agents defined.


In [6]:
# @title 6. Define Tasks for Learning Path Crew

print("Defining Learning Path Tasks...")

# Verify agents are defined
if 'goal_clarifier' not in locals() or \
   'knowledge_assessor' not in locals() or \
   'resource_finder' not in locals() or \
   'path_planner' not in locals() or \
   'plan_presenter' not in locals():
    raise ValueError("One or more required agents are not defined.")

# --- Task 1: Clarify Goal into Sub-topics ---
task_clarify_goal = Task(
    description=(
        f"Analyze the user's primary learning goal: '{learning_goal}'. "
        f"Break this goal down into 3 to 5 core sub-topics or essential skill areas that someone needs to master to achieve this goal. "
        f"Present these sub-topics as a clear, numbered list."
    ),
    agent=goal_clarifier,
    expected_output=(
        "A numbered list containing 3-5 key sub-topics or skill areas derived from the main learning goal."
        " Example: '1. Python Fundamentals\n2. Introduction to Pandas...'."
    )
)
print("- Task 'task_clarify_goal' defined.")

# --- Task 2: Assess Knowledge Gaps ---
task_assess_knowledge = Task(
    description=(
        f"Review the user's self-described current knowledge level: '{current_knowledge}'. "
        f"Compare this against the list of required sub-topics derived in the previous task (context will be provided). "
        f"Identify and clearly state the main knowledge gaps, focusing on which of the required sub-topics the user likely needs to learn from scratch or review significantly."
    ),
    agent=knowledge_assessor,
    expected_output=(
        "A concise summary identifying the primary knowledge gaps. This should explicitly mention which of the required sub-topics (from Task 1) represent the biggest learning needs for the user based on their background."
        " Example: 'Based on the user's JavaScript background but no Python experience, the main gaps are in Python Fundamentals, Pandas, NumPy, and Scikit-learn basics...'"
    ),
    context=[task_clarify_goal] # Needs the sub-topics list
)
print("- Task 'task_assess_knowledge' defined.")

# --- Task 3: Find Learning Resources ---
task_find_resources = Task(
    description=(
        f"For each core sub-topic identified in Task 1 (list provided in context), use web search tools to find 1 or 2 high-quality online learning resources. "
        f"Strongly prioritize resources that match the user's preferred learning style: '{preferred_style}'. "
        f"Resources should be specific URLs (e.g., to a tutorial, video playlist, interactive course, documentation section, project repository). "
        f"Organize the findings clearly, grouping resources by the sub-topic they address."
    ),
    agent=resource_finder,
    expected_output=(
        "A structured list of recommended learning resources, grouped by sub-topic. Each entry should include the resource name/description and a direct URL. "
        "Example:\nSub-Topic: Python Fundamentals\n- Resource: [Name of Video Tutorial Series] - [URL]\n- Resource: [Name of Interactive Platform] - [URL]\nSub-Topic: Pandas Basics\n- ..."
    ),
    context=[task_clarify_goal], # Needs the sub-topics list
    tools=available_tools # Explicitly mention tools if needed
)
print("- Task 'task_find_resources' defined.")

# --- Task 4: Plan Learning Path ---
task_plan_path = Task(
    description=(
        f"Design a logical step-by-step learning path using the identified sub-topics (from Task 1), the assessment of knowledge gaps (from Task 2), and the curated resources (from Task 3). "
        f"The path should start by addressing the main knowledge gaps. Sequence the sub-topics and their associated resources in a progressive order that makes sense for a beginner aiming for the goal: '{learning_goal}'. "
        f"Consider prerequisite relationships between topics. You can suggest logical phases or milestones."
    ),
    agent=path_planner,
    expected_output=(
        "A structured learning path presented as a sequence of steps or phases. Each step should name the sub-topic(s) to focus on and refer to the specific resources (identified in Task 3) recommended for that step. "
        "The sequence should be logical for the learner's background and goal."
        " Example:\nPhase 1: Python Basics\n- Focus on: Python Fundamentals\n- Resources: [Link 1], [Link 2]\nMilestone: Complete basic syntax exercises.\nPhase 2: Data Handling\n- Focus on: Pandas Basics, NumPy Introduction\n- Resources: [Link 3], [Link 4]..."
    ),
    # Needs all previous outputs to create the plan
    context=[task_clarify_goal, task_assess_knowledge, task_find_resources]
)
print("- Task 'task_plan_path' defined.")

# --- Task 5: Present Final Plan ---
task_present_plan = Task(
    description=(
        f"Take the sequenced learning path designed in the previous task (context provided) and format it into a final, user-friendly document. "
        f"Use clear headings for each step/phase, bullet points for resources, and include the direct URLs provided by the Resource Finder. "
        f"Add a brief introductory sentence acknowledging the learning goal and a short concluding sentence to encourage the user. "
        f"Ensure the final output is well-organized, actionable, and easy to read."
    ),
    agent=plan_presenter,
    expected_output=(
        "A polished and clearly formatted learning plan document. It should include a brief intro, numbered steps or phases with clear titles, bullet points listing recommended resources with URLs for each step, and potentially a brief concluding remark. The formatting should enhance readability (e.g., using Markdown)."
    ),
    context=[task_plan_path] # Needs the final structured plan
)
print("- Task 'task_present_plan' defined.")

print("All learning path tasks defined.")

Defining Learning Path Tasks...
- Task 'task_clarify_goal' defined.
- Task 'task_assess_knowledge' defined.
- Task 'task_find_resources' defined.
- Task 'task_plan_path' defined.
- Task 'task_present_plan' defined.
All learning path tasks defined.


In [7]:
# @title 7. Create and Run Learning Path Crew

print("Creating the Learning Path Crew...")

# Verify all components are ready
if 'goal_clarifier' not in locals() or \
   'knowledge_assessor' not in locals() or \
   'resource_finder' not in locals() or \
   'path_planner' not in locals() or \
   'plan_presenter' not in locals() or \
   'task_clarify_goal' not in locals() or \
   'task_assess_knowledge' not in locals() or \
   'task_find_resources' not in locals() or \
   'task_plan_path' not in locals() or \
   'task_present_plan' not in locals():
    raise ValueError("Missing agents or tasks needed to create the Learning Path Crew.")

# Assemble the Crew
learning_path_crew = Crew(
    agents=[goal_clarifier, knowledge_assessor, resource_finder, path_planner, plan_presenter],
    tasks=[task_clarify_goal, task_assess_knowledge, task_find_resources, task_plan_path, task_present_plan],
    process=Process.sequential,
    memory=True,
    cache=True,
    verbose=True
)

print("Learning Path Crew created. Starting the process with kickoff()...")

# Define initial inputs (can be empty if all info is in prompts/task descriptions)
# Passing them here can also work if tasks are designed to use `inputs[...]`
crew_inputs = {
    'learning_goal': learning_goal,
    'current_knowledge': current_knowledge,
    'preferred_style': preferred_style
}

# Kick off the crew's work
# This will take time and API calls
result = learning_path_crew.kickoff(inputs=crew_inputs)

print("\n\n*****************************************")
print("   LEARNING PATH CREW FINISHED WORK!   ")
print("*****************************************")

Creating the Learning Path Crew...
Learning Path Crew created. Starting the process with kickoff()...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 88e005ee-f382-4a7a-ad60-70501bcd7298                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 26f9c5ff-e1ab-4b68-b902-b7cd7a7eeb4f
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 26f9c5ff-e1ab-4b68-b902-b7cd7a7eeb4f
       Status: Executing Task...
    └── 🤖 Agent: Learning Goal Analyst
            Status: In Progress

# Agent: Learning Goal Analyst
## Task: Analyze the user's primary learning goal: 'Learn Python programming for data analysis and basic machine learning tasks.'. Break this goal down into 3 to 5 core sub-topics or essential skill areas that someone needs to master to achieve this goal. Present these sub-topics as a clear, numbered list.


...

🚀 Crew: crew
├── 📋 Task: 26f9c5ff-e1ab-4b68-b902-b7cd7a7eeb4f
│      Assigned to: Learning Goal Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Learning Goal Analyst
│           Status: ✅ Completed
├── 📋 Task: 6b28a820-4456-48e0-a7f9-647a152c99ec
│      Assigned to: Skill Assessment Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Skill Assessment Specialist
│           Status: ✅ Completed
├── 📋 Task: dff0b9f1-e006-49cc-a589-105321d3d253
│      Assigned to: Educational Resource Curator
│      Status: ✅ Completed
│   └── 🤖 Agent: Educational Resource Curator
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (2)
├── 📋 Task: b87e07a3-d2bc-475a-8d68-0b6de578e0df
│      Assigned to: Learning Path Designer
│      Status: ✅ Completed
│   └── 🤖 Agent: Learning Path Designer
│           Status: ✅ Completed
└── 📋 Task: 3ae304c3-ee55-4f86-b73d-71032b5fe6d2
       Status: Executing Task...
    └── 🤖 Agent: Learning Plan Communicator
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 26f9c5ff-e1ab-4b68-b902-b7cd7a7eeb4f
│      Assigned to: Learning Goal Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Learning Goal Analyst
│           Status: ✅ Completed
├── 📋 Task: 6b28a820-4456-48e0-a7f9-647a152c99ec
│      Assigned to: Skill Assessment Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Skill Assessment Specialist
│           Status: ✅ Completed
├── 📋 Task: dff0b9f1-e006-49cc-a589-105321d3d253
│      Assigned to: Educational Resource Curator
│      Status: ✅ Completed
│   └── 🤖 Agent: Educational Resource Curator
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (2)
├── 📋 Task: b87e07a3-d2bc-475a-8d68-0b6de578e0df
│      Assigned to: Learning Path Designer
│      Status: ✅ Completed
│   └── 🤖 Agent: Learning Path Designer
│           Status: ✅ Completed
└── 📋 Task: 3ae304c3-ee55-4f86-b73d-71032b5fe6d2
       Assigned to: Learning Plan Communicator
       Status: ✅ Completed
    └── 🤖 Agent: Learning Plan Communicator
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3ae304c3-ee55-4f86-b73d-71032b5fe6d2                                                                     │
│  Agent: Learning Plan Communicator                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 88e005ee-f382-4a7a-ad60-70501bcd7298                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



*****************************************
   LEARNING PATH CREW FINISHED WORK!   
*****************************************


In [8]:
# @title 8. Display Final Learning Plan

print("\n\n#####################################")
print("## Generated Personalized Learning Plan:")
print("#####################################\n")

if 'result' in locals():
    # Print the final output from the 'plan_presenter' agent
    print(result)
else:
    print("!!! The 'result' variable is not defined. Ensure Cell 7 executed correctly.")



#####################################
## Generated Personalized Learning Plan:
#####################################

### **Welcome to Your Learning Path: Python Programming for Data Analysis and Basic Machine Learning**
Congratulations on taking the first step towards mastering Python for data analysis and machine learning! This structured learning path is tailored to transition you from Python basics to handling real-world data science projects. Let's embark on this exciting journey!

---
#### **Phase 1: Python Basics**
Learn the core concepts of Python to lay a strong foundation for more advanced topics.
- **Objective:** Master basic Python syntax, data types, functions, and control structures.
- **Resources:**
  - [9 HOURS of Python Projects - From Beginner to Advanced (YouTube)](https://www.youtube.com/watch?v=NpmFbWO6HPU)
  - [12 Beginner Python Projects - Coding Course (YouTube)](https://www.youtube.com/watch?v=8ext9G7xspg)
- **Milestone:** Successfully write and debug basic P